# Contraste de Hipótesis entre Modelos de Machine Learning en Julia

## Introducción
En este notebook, no pretende ser un manual completo de como realizar un contraste de hipotesis entre diferentes modelos de machine learning. Lo que se puede encontrar es una comparativa de rendimiento de dos modelos. En concreto y para el notebook sea autocontenido se va a emplear una librería bastante conocida para la creación de los modelos como es MLJ y otra que nos dará el apoyo necesario para la ejecución de los contrastes. Tenga en cuenta que lo mismo se puede hacer con Scikit-Learn siempre que se saquen las medidas de rendimiento. 

## Configuración del entorno
Primero de todo vamos a instalar las librerías que nos van a hacer falta


In [1]:
using Pkg;
Pkg.add(["DataFrames", "CSV", "MLJ", "Random", "HypothesisTests", "Statistics", "DecisionTree", "MLJDecisionTreeInterface", "NearestNeighborModels"])
using DataFrames, CSV, MLJ, Random, HypothesisTests, Statistics


    Updating registry at `C:\Users\Alberto S\.julia\registries\General.toml`
   Resolving package versions...
   Installed HypergeometricFunctions ── v0.3.25
   Installed PDMats ─────────────────── v0.11.31
   Installed StatsFuns ──────────────── v1.3.2
   Installed StatisticalTraits ──────── v3.4.0
   Installed CategoricalDistributions ─ v0.1.15
   Installed LearnAPI ───────────────── v0.1.0
   Installed NearestNeighborModels ──── v0.2.3
   Installed Roots ──────────────────── v2.2.1
   Installed EarlyStopping ──────────── v0.3.0
   Installed DataFrames ─────────────── v1.7.0
   Installed PtrArrays ──────────────── v1.2.1
   Installed MLJFlow ────────────────── v0.2.0
   Installed IterationControl ───────── v0.5.4
   Installed MLJModels ──────────────── v0.16.17
   Installed StatisticalMeasures ────── v0.1.7
   Installed MLJIteration ───────────── v0.6.3
   Installed HypothesisTests ────────── v0.11.3
   Installed Rmath_jll ──────────────── v0.5.1+0
   Installed MLJBase ──────────────

## Cargar y preparar los datos
El siguiente paso es la descarga y definición del problema que se va a emplear, en este caso, se empleará un sencillo problema de clasificación binario.


In [2]:

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
data = CSV.File(download(url), header=false) |> DataFrame

# Renombramos las columnas para mayor claridad
rename!(data, [:num_embarazos, :glucosa, :presion_arterial, :pliegue_cutaneo, :insulina, :indice_masa_corporal, :historia_familiar, :edad, :diabetes])
first(data, 5)  # Mostrar las primeras 5 filas del dataset


Row,num_embarazos,glucosa,presion_arterial,pliegue_cutaneo,insulina,indice_masa_corporal,historia_familiar,edad,diabetes
,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1


## División de los datos en entrenamiento y prueba
A continuación se preparan los datos repartiendolos entre entrenamiento y test.


In [3]:
using Random

Random.seed!(42)
train, test = partition(eachindex(data.diabetes), 0.7, shuffle=true)
X_train = data[train, Not(:diabetes)]
y_train = categorical(data[train, :diabetes])
X_test = data[test, Not(:diabetes)]
y_test = categorical(data[test, :diabetes])

println("El conjunto inicial es ", size(data), " mientras que el conjunto de entrenamiento es ",size(X_train),"->",size(y_train),
    " y el de test es ", size(X_test),"->",size(y_test) 
)

El conjunto inicial es (768, 9) mientras que el conjunto de entrenamiento es (538, 8)->(538,) y el de test es (230, 8)->(230,)



## Entrenamiento de los modelos
En este caso se van a entrenar dos modelos básicos para comparar su rendimiento el KNN y un árbol de decisión.


In [4]:

DecisionTree = @load DecisionTreeClassifier pkg=DecisionTree
KNN = @load KNNClassifier pkg=NearestNeighborModels

tree_model = DecisionTree(max_depth=5)
knn_model = KNN(K=3)

tree_machine = machine(tree_model, X_train, y_train)
fit!(tree_machine)
knn_machine = machine(knn_model, X_train, y_train)
fit!(knn_machine)

import MLJDecisionTreeInterface

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Alberto S\.julia\packages\MLJModels\ziReN\src\loading.jl:159


 ✔
import NearestNeighborModels

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Alberto S\.julia\packages\MLJModels\ziReN\src\loading.jl:159


 ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = 5, …), …).
└ @ MLJBase C:\Users\Alberto S\.julia\packages\MLJBase\7nGJF\src\machines.jl:499
┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc NearestNeighborModels.KNNClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(m

trained Machine; caches model-specific representations of data
  model: KNNClassifier(K = 3, …)
  args: 
    1:	Source @147 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}
    2:	Source @017 ⏎ AbstractVector{Multiclass{2}}


## Evaluación de los modelos
Evaluaremos el rendimiento de ambos modelos en el conjunto de prueba utilizando cross-validation.


In [5]:
# Cross-validation para Árbol de Decisión
y_pred_tree = predict_mode(tree_machine, X_test)
# Cross-validation para k-NN
y_pred_knn = predict_mode(knn_machine, X_test)


tree_results = y_pred_tree .== y_test
knn_results = y_pred_knn .== y_test

# Medir precisión en el conjunto de prueba
accuracy_tree = mean(tree_results)
accuracy_knn = mean(knn_results)

println("Precisión en el conjunto de prueba del Árbol de Decisión: ", accuracy_tree)
println("Precisión en el conjunto de prueba del k-NN: ", accuracy_knn)

Precisión en el conjunto de prueba del Árbol de Decisión: 0.7521739130434782
Precisión en el conjunto de prueba del k-NN: 0.6782608695652174


Si bien estos datos están bien, no nos dan una idea de como de mejor es uno de los modelos respecto del otro. Para poder asegurar con unas ciertas garantias tal cosa es necesario hacer un contraste de hipótesis, para lo cual hace falta más de un experimento como en un cross_validation de 10 como en este caso.

In [6]:
X = data[:, Not(:diabetes)]
y = categorical(data[:, :diabetes])

tree_machine = machine(tree_model, X, y)
knn_machine = machine(tree_model, X, y)

results_tree = evaluate!(tree_machine, resampling=CV(nfolds=5, shuffle=true, rng=42),
          measure= [Accuracy()])

results_knn = evaluate!(knn_machine, resampling=CV(nfolds=5, shuffle=true, rng=42),
          measure= [Accuracy()])

println("Los resultados para cada partición son:
    DT = ", results_tree.per_fold, "
    KNN= ", results_knn. per_fold)

Evaluating over 5 folds: 100%[=========================] Time: 0:00:03


Los resultados para cada partición son:
    DT = [[0.7337662337662337, 0.7467532467532467, 0.7467532467532467, 0.738562091503268, 0.7450980392156863]]
    KNN= [[0.7272727272727273, 0.7402597402597403, 0.7467532467532467, 0.738562091503268, 0.7450980392156863]]


## Contraste de hipótesis

El rendimiento de comparar varios modelos de machine learning utilizando tanto un test t de Student como un test de Mann-Whitney. Usaremos un conjunto de datos de ejemplo y dos modelos de clasificación. El objetivo es determinar si hay una diferencia significativa en el rendimiento de los modelos.

### Test t de Student
Compararemos las medias de las exactitudes de los dos modelos usando un test t.

In [7]:
using HypothesisTests;
t_test_result = OneSampleTTest(results_tree.per_fold[1], results_knn.per_fold[1])
println("Resultado del test: p-valor = ", pvalue(t_test_result))

Resultado del test: p-valor = 0.17780780835622115


En este caso nos diría que ambas medias son iguales. 

### Test de Mann-Whitney
Este test no paramétrico se utiliza para comparar muestras independientes. Compara las medianas y no es paramétrico, es decir no asume la normalidad en las distribuciones. En este caso es aun menos potente en la detecciónd e diferencias pero nos servirá para ver ejemplo


In [8]:
kruskal_result = ApproximateMannWhitneyUTest(results_tree.per_fold[1], results_knn.per_fold[1])
println("Resultado del test: p-valor = ", pvalue(kruskal_result))

Resultado del test: p-valor = 0.6704021525148367
